<a href="https://colab.research.google.com/github/AlexKressner/OR_Workshop/blob/main/Verteilung_Seminarthemen.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Verteilung Seminarthemen

In [41]:
# Notwendigen Programminstallationen
# pip als Paketmanager
!pip install -U -q pip
!pip install -q ortools
# Laden des Programms
from ortools.linear_solver import pywraplp

In [42]:
# Solver mit SCIP als Backend.
# SCIP implementiert Simplex, Branch-and-Bound, etc
solver = pywraplp.Solver.CreateSolver('SCIP')

## Indexmengen

In [43]:
I = ['S1','S2','S3','S4'] # Menge der Studierenden
J = ['B1','B2','B3'] # Menge der Betreuer

In [44]:
J

['B1', 'B2', 'B3']

## Entscheidungsvariablen

In [45]:
infinity = solver.infinity()

In [46]:
# Definition der Entscheidungsvariablen
X={}
for i in I:
  for j in J: 
      X[i,j] = solver.BoolVar(f"{i},{j}")
print('Anzahl Entscheidungsvariablen =', solver.NumVariables())

Anzahl Entscheidungsvariablen = 12


## Parameter

In [47]:
p={} # Präferenzen
p['S1']={'B1':2, 'B2':9, 'B3':7}
p['S2']={'B1':2, 'B2':10, 'B3':7}
p['S3']={'B1':4, 'B2':10, 'B3':8}
p['S4']={'B1':6, 'B2':9, 'B3':9}

In [48]:
b={} # Kapazitäten
b['B1']=2
b['B2']=1
b['B3']=2

## Zielfunktion

In [49]:
p

{'S1': {'B1': 2, 'B2': 9, 'B3': 7},
 'S2': {'B1': 2, 'B2': 10, 'B3': 7},
 'S3': {'B1': 4, 'B2': 10, 'B3': 8},
 'S4': {'B1': 6, 'B2': 9, 'B3': 9}}

In [50]:
# Maximierung der gesamten Deckungsbeiträge
solver.Maximize(sum(p[i][j]*X[i,j] for i in I for j in J))

## Nebenbedingungen

In [51]:
for i in I:
    solver.Add(sum(X[i,j] for j in J)==1)

In [52]:
for j in J:
    solver.Add(sum(X[i,j] for i in I)<=b[j])
print('Anzahl Nebenbedingungen =', solver.NumConstraints())

Anzahl Nebenbedingungen = 7


## Berechnung Lösung

In [53]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('LÖSUNG:')
    print('Zielfunktionswert (Präferenz) =', solver.Objective().Value())
    for i in I:
      for j in J:
          print(f'{j} =', round(X[i,j].solution_value()))
else:
    print('Problem hat keine Lösung')

LÖSUNG:
Zielfunktionswert (Präferenz) = 31.0
B1 = 0
B2 = 0
B3 = 1
B1 = 0
B2 = 1
B3 = 0
B1 = 0
B2 = 0
B3 = 1
B1 = 1
B2 = 0
B3 = 0
